<a href="https://www.kaggle.com/code/mdnahidulislam2005/sales-prediction?scriptVersionId=293162869" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [123]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-business-analysis-20222024/ecommerce_sales_data.csv


In [124]:
df = pd.read_csv("/kaggle/input/e-commerce-business-analysis-20222024/ecommerce_sales_data.csv")

In [125]:
df .head()

,Order Date,Product Name,Category,Region,Quantity,Sales,Profit
0,2024-12-31,Printer,Office,North,4,3640,348.93
1,2022-11-27,Mouse,Accessories,East,7,1197,106.53
2,2022-05-11,Tablet,Electronics,South,5,5865,502.73
3,2024-03-16,Mouse,Accessories,South,2,786,202.87
4,2022-09-10,Mouse,Accessories,West,1,509,103.28


In [126]:
df['Category'].value_counts()

Category
Electronics    1742
Accessories    1401
Office          357
Name: count, dtype: int64

In [127]:
df.isnull().sum()

Order Date      0
Product Name    0
Category        0
Region          0
Quantity        0
Sales           0
Profit          0
dtype: int64

In [128]:
category_grouped = df.groupby('Category')['Sales'].value_counts()


In [129]:
category_grouped

Category     Sales
Accessories  3186     5
             2640     4
             312      3
             384      3
             416      3
                     ..
Office       9387     1
             9459     1
             9477     1
             9528     1
             10656    1
Name: count, Length: 2943, dtype: int64

In [130]:
x = df.drop(['Sales'],axis = 1)
y = df['Sales']

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn .pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [132]:
categorical_cols = x.select_dtypes(include=['object']).columns.tolist()

In [133]:
numerical_cols = x.select_dtypes(include=['int64','float']).columns.tolist()

In [134]:
numerical_transformer = Pipeline(steps=[
                        ('imputer',SimpleImputer(strategy='mean')),
                        ('scaler',StandardScaler())    
])

In [135]:
categorical_transformer = Pipeline(steps=[
   ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [136]:
preprocessor = ColumnTransformer(transformers=[
                              ('num',numerical_transformer,numerical_cols),
                               ('cat',categorical_transformer,categorical_cols )
])

In [138]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [139]:
from sklearn.linear_model import LinearRegression

In [140]:
model_1 = Pipeline(steps=[
                 ('pre',preprocessor),('reg',LinearRegression())
])

In [141]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [142]:
model_1.fit(X_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Quantity', 'Profit']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Order Date', 'Product Name',
                                                   'Category', 'Region'])])),
                ('reg', LinearRegression())])

In [143]:
model_prediction_1 = model.predict(X_test)

print(f'Accuracy:{r2_score(y_test,model_prediction_1):.4f}')

Accuracy:0.6079


In [144]:
from sklearn.tree import DecisionTreeRegressor

In [145]:
model_2 = Pipeline(steps=[
           ('pre',preprocessor),('reg',DecisionTreeRegressor(max_depth=7,random_state=42))
])

In [146]:
model_2.fit(X_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Quantity', 'Profit']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Order Date', 'Product Name',
                                                   'Category', 'Region'])])),
                ('reg', DecisionTreeRegressor(max_depth=7, random_state=42))])

In [147]:
model_prediction_2 = model_2.predict(X_test)


print(f'Accuracy:{r2_score(model_prediction_2,y_test):.4f}')

Accuracy:0.6587


In [148]:
from xgboost import XGBRegressor

In [149]:
model_3 = Pipeline(steps=[
          ('pre',preprocessor),('reg',XGBRegressor(n_estimators=100,learning_rate=0.1,random_state=42))
])

In [150]:
model_3.fit(X_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Quantity', 'Profit']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Order Date', 'Product Name',
                                                   'Category', 'Region'])])),
                ('reg',
                 XGBRe...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [151]:
model_prediction_3=model_3.predict(X_test)


print(f'Accuracy:{r2_score(model_prediction_3,y_test):.4f}')

Accuracy:0.6566


In [152]:
from sklearn.ensemble import RandomForestRegressor

In [153]:
model_4 = Pipeline(steps=[
                    ('pre',preprocessor),('reg',RandomForestRegressor(n_estimators=100,random_state = 42))
])

In [154]:
model_4.fit(X_train,y_train)
model_prediction_4 = model_4.predict(X_test)


print(f'Accuracy{r2_score(model_prediction_4,y_test):.4f}')

Accuracy0.5994


i am compare 4 model and i have got 65% acuracy.I am try my best.